In [11]:
import sys
import numpy as np
from functools import reduce

from pyscf.lib import logger
from pyscf import lib

from lno.base import LNO
from lno.afqmc import LNOAFQMC
from lno.afqmc import mpi_jax

from pyscf import gto, scf, mp, cc

log = logger.Logger(sys.stdout, 6)

In [6]:
atom = '''
 O   -1.485163346097   -0.114724564047    0.000000000000
 H   -1.868415346097    0.762298435953    0.000000000000
 H   -0.533833346097    0.040507435953    0.000000000000
 O    1.416468653903    0.111264435953    0.000000000000
 H    1.746241653903   -0.373945564047   -0.758561000000
 H    1.746241653903   -0.373945564047    0.758561000000
'''

basis = 'sto6g'

mol = gto.M(atom=atom, basis=basis)
mol.verbose = 4

mf = scf.RHF(mol).density_fit()
mf.kernel()



frozen = 2
    

## canonical
mmp = mp.MP2(mf, frozen=frozen)
mmp.kernel()
#
mcc = cc.CCSD(mf, frozen=frozen).set(verbose=5)
eris = mcc.ao2mo()
mcc.kernel(eris=eris)
from pyscf.cc.ccsd_t import kernel as CCSD_T
eccsd_t = CCSD_T(mcc, eris)
filename = 'fragmentenergies.txt'



******** <class 'pyscf.df.df_jk.DFRHF'> ********
method = DFRHF
initial guess = minao
damping factor = 0
level_shift factor = 0
DIIS = <class 'pyscf.scf.diis.CDIIS'>
diis_start_cycle = 1
diis_space = 8
diis_damp = 0
SCF conv_tol = 1e-09
SCF conv_tol_grad = None
SCF max_cycles = 50
direct_scf = False
chkfile to save SCF result = /tmp/tmpbgmlaknz
max_memory 4000 MB (current use 158 MB)
Set gradient conv threshold to 3.16228e-05
Initial guess from minao.
******** <class 'pyscf.df.df.DF'> ********
auxbasis = None
max_memory = 4000
l = 0, exps = 0.200224856 * 2^n for n = 0..8
l = 0, exps = 0.491761212 * 2^n for n = 0..12
l = 1, exps = 0.491761212 * 2^n for n = 0..10
l = 2, exps = 0.491761212 * 2^n for n = 0..7

WARN: Even tempered Gaussians are generated as DF auxbasis for  H O

init E= -151.176325562043
  HOMO = -0.417601638291787  LUMO = 0.371378128331864
cycle= 1 E= -151.274606807763  delta_E= -0.0983  |g|= 0.49  |ddm|= 2.39
  HOMO = -0.23592395822799  LUMO = 0.597817788208578
cycle= 2

<class 'pyscf.cc.dfccsd.RCCSD'> does not have attributes  converged


    CPU time for RCCSD iter      0.08 sec, wall time      0.08 sec
cycle = 3  E_corr(RCCSD) = -0.0997077570096607  dE = -0.00326501285  norm(t1,t2) = 0.00578255
    CPU time for RCCSD iter      1.36 sec, wall time      0.20 sec
cycle = 4  E_corr(RCCSD) = -0.0997680543388238  dE = -6.02973292e-05  norm(t1,t2) = 0.000916771
    CPU time for RCCSD iter      0.08 sec, wall time      0.09 sec
cycle = 5  E_corr(RCCSD) = -0.0997509610045389  dE = 1.70933343e-05  norm(t1,t2) = 0.000264224
    CPU time for RCCSD iter      0.17 sec, wall time      0.10 sec
cycle = 6  E_corr(RCCSD) = -0.0997504788057533  dE = 4.82198786e-07  norm(t1,t2) = 5.32544e-05
    CPU time for RCCSD iter      0.06 sec, wall time      0.09 sec
cycle = 7  E_corr(RCCSD) = -0.0997498384409597  dE = 6.40364794e-07  norm(t1,t2) = 1.28645e-05
    CPU time for RCCSD iter      0.97 sec, wall time      0.15 sec
cycle = 8  E_corr(RCCSD) = -0.0997497409458718  dE = 9.74950879e-08  norm(t1,t2) = 4.36706e-06
    CPU time for RCCSD iter 

In [14]:
import sys
print(sys.path)

['/home/yichi/research/lno_afqmc/ad_afqmc', '/home/yichi/research/lno_afqmc/lno_new', '/home/yichi/research/pyscf', '/home/yichi/research', '/home/yichi/miniconda3/envs/lno_afqmc/lib/python311.zip', '/home/yichi/miniconda3/envs/lno_afqmc/lib/python3.11', '/home/yichi/miniconda3/envs/lno_afqmc/lib/python3.11/lib-dynload', '', '/home/yichi/miniconda3/envs/lno_afqmc/lib/python3.11/site-packages', '/home/yichi/miniconda3/envs/lno_afqmc/lib/python3.11/site-packages/setuptools/_vendor']


In [7]:
#vmc_root = '/projects/joku8258/software/alpine_software/Dice'
# LNO
for thresh in [1e-4]:
    f = open(filename,'w')
    f.close()
    mfcc = LNOAFQMC(mf, thresh=thresh, frozen=frozen).set(verbose=5)
    mfcc.thresh_occ = thresh*10
    mfcc.thresh_vir = thresh
    mfcc.nblocks = 20
    mfcc.nwalk_per_proc = 10#10
#    mfcc.seed = 1234 
#    mfcc.runfrags=[4]
    mfcc.lo_type = 'pm'
    mfcc.no_type = 'cim'
    mfcc.frag_lolist = '1o'
    mfcc.force_outcore_ao2mo = True
#    mfcc.vmc_root = vmc_root
    mfcc.kernel()#canonicalize=False,chol_vecs=chol_vecs)
    ecc = mfcc.e_corr #+ mf.energy_nuc()
    ecc_pt2corrected = mfcc.e_corr_pt2corrected(mmp.e_corr)
    #ecc_t = mfcc.e_corr_ccsd_t
#    log.info('thresh = %.0e  E_corr(AFQMC)     = %.10f, %.10f  rel(wrt CCSD) = %6.2f%%  '
#             'diff = % .10f',
#             thresh, ecc,mf.energy_nuc(), ecc/mcc.e_corr*100, ecc-mcc.e_corr)
    log.info('thresh = %.0e  E_corr(AFQMC)     = %.10f, %.10f  rel(wrt CCSD) = %6.2f%%  '
             'diff = % .10f',
             thresh, ecc,mf.energy_nuc(), 0, 0)
    log.info('                E_corr(AFQMC+PT2) = %.10f  rel = %6.2f%%  '
                 'diff = % .10f',
                 ecc_pt2corrected, ecc_pt2corrected/mcc.e_corr*100,
                 ecc_pt2corrected-mcc.e_corr)


******** <class 'lno.afqmc.afqmc.LNOAFQMC'> ********
nocc = 8, nmo = 12
frozen orbitals 2
max_memory 4000 MB (current use 169 MB)
thresh_occ = 1.000000e-03
thresh_vir = 1.000000e-04
force_outcore_ao2mo = True
no_type = ie
Constructing LOs
lo_type = pm
LOs span exactly the occupied space.
Using single-LO fragment
nfrag = 8  nlo = 8
frag_atmlist = None
frag_lolist = [[0], [1], [2], [3], [4], [5], [6], [7]]
frag_wghtlist = [1. 1. 1. 1. 1. 1. 1. 1.]
frag_nonvlist = None
    CPU time for LO and fragment       20.88 sec, wall time      2.69 sec
ao2mo est mem= 0.10 MB  avail mem= 3813.90 MB
Lov is saved to /tmp/nkjdiams
    CPU time for transforming DF-MP2 integrals      0.02 sec, wall time      0.00 sec
    CPU time for Integral xform         0.11 sec, wall time      0.02 sec
    CPU time for make pno               0.00 sec, wall time      0.00 sec
Solving impurity problem
DQMC root:  /projects/joku8258/software/VMC_forwork/VMC/
#
# Preparing AFQMC calculation
# Calculating Cholesky integra